In [1]:
!apt install python3.10-venv
!python -m venv audio_env
!source activate audio_env/bin/activate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 2,473 kB of archives.
After this operation, 2,882 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.3 [1,679 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.1 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.2 [5,724 B]
Fetched 2,473 kB in 3s (969 kB/s)
Selecting previously unselected package python3-pip-whl.
(Reading database ... 120500 files and directories currently installed.)
Prep

In [2]:
!pip install datasets
!pip install wandb
!pip install transformers==4.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=d1c33a9ce95b1c6113e3094e36a4fadc6baf8827ac86c702fe3ab40e72334ac6
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8

In [3]:
import os
import logging
import librosa

import wandb
import numpy as np

from datasets import DatasetDict, load_dataset, load_metric
from transformers import (
    HubertForSequenceClassification,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    Wav2Vec2FeatureExtractor,
    HubertConfig,
)

In [4]:
model = "facebook/hubert-base-ls960"
feature_extractor = Wav2Vec2FeatureExtractor(
          sampling_rate=16000,
          truncation=True
)
print(feature_extractor)

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000,
  "truncation": true
}



In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# model_id = "facebook/hubert-base-ls960"
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_id)
ds_path = "/content/drive/MyDrive/Data/Diss/train/cv-valid-train/"
audio_file = ds_path + "0/sample-000030.mp3"
audio_array = librosa.load(audio_file, sr=16000, mono=False)[0]
input = feature_extractor(
       raw_speech=audio_array,
       sampling_rate=16000,
       padding=True,
      return_tensors="pt"
)
print(input)
#print(librosa.get_samplerate(audio_file))
print(input.input_values.shape)
print(audio_array.shape)

{'input_values': tensor([[-0.0011, -0.0011, -0.0011,  ..., -0.0008, -0.0010, -0.0012]])}
torch.Size([1, 82560])
(82560,)


In [7]:
audio_file_1 = ds_path + "0/sample-000031.mp3"
audio_file_2 = ds_path + "0/sample-000032.mp3"
audio_array_1 = librosa.load(audio_file_1, sr=16000, mono=False)[0]
audio_array_2 = librosa.load(audio_file_2, sr=16000, mono=False)[0]
input_with_one_audio = feature_extractor(
       audio_array_1,
       sampling_rate=16000,
       padding=True,
      return_tensors="pt"
)
input_with_two_audios = feature_extractor(
       [audio_array_1, audio_array_2],
       sampling_rate=16000,
       padding=True,
      return_tensors="pt"
)
print(input_with_one_audio.input_values.shape)
print(input_with_two_audios.input_values.shape)

torch.Size([1, 123648])
torch.Size([2, 142848])


In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualizing data
import seaborn as sns # visualizing data with stunning default theme
import sklearn # contain algorithms
import warnings
warnings.filterwarnings('ignore')

# load dataset from input directory
df = pd.read_csv("/content/drive/MyDrive/Data/Diss/cv-valid-train.csv")
df[df['age'].notna()].head()
df =  df.head(int(len(df)*0.01))
print(len(df))
df.tail()

1957


,filename,text,up_votes,down_votes,age,gender,accent,duration
1952,cv-valid-train/sample-001952.mp3,a boy came running towards me,2,0,NaN,NaN,NaN,NaN
1953,cv-valid-train/sample-001953.mp3,the usb device didn't function correctly,1,0,fifties,male,australia,NaN
1954,cv-valid-train/sample-001954.mp3,we're a different world,1,0,fourties,male,us,NaN
1955,cv-valid-train/sample-001955.mp3,you must love the desert but never trust it co...,4,0,NaN,NaN,NaN,NaN
1956,cv-valid-train/sample-001956.mp3,he was immensely excited at the news,2,0,NaN,NaN,NaN,NaN


In [9]:
#we extract the columns that we think useful are
df = df[['filename','age','gender']]
#To clean the data we remove the sample with NaN attribute values.
data = df[df['age'].notna() & df['gender'].notna()]
data.reset_index(inplace=True, drop=True)
data.head()

,filename,age,gender
0,cv-valid-train/sample-000005.mp3,twenties,female
1,cv-valid-train/sample-000008.mp3,seventies,male
2,cv-valid-train/sample-000013.mp3,thirties,female
3,cv-valid-train/sample-000014.mp3,sixties,male
4,cv-valid-train/sample-000019.mp3,fifties,male


In [10]:
#if the below code fail to covert gender datatype and values then uncomment above code
cleanup_nums = {"gender": {"male":1,"female":0,"other":0.5}}
data = data.replace(cleanup_nums)
data.head()

,filename,age,gender
0,cv-valid-train/sample-000005.mp3,twenties,0.0
1,cv-valid-train/sample-000008.mp3,seventies,1.0
2,cv-valid-train/sample-000013.mp3,thirties,0.0
3,cv-valid-train/sample-000014.mp3,sixties,1.0
4,cv-valid-train/sample-000019.mp3,fifties,1.0


In [11]:
 print(len(data['age'].unique()) )

8


In [12]:
NUM_LABELS = len(data['age'].unique())
model_id = "facebook/hubert-base-ls960"

config = HubertConfig.from_pretrained(model_id, num_labels=NUM_LABELS)
hubert_model = HubertForSequenceClassification.from_pretrained(
    model_id,
    config=config,  # because we need to update num_labels as per our dataset
    ignore_mismatched_sizes=True,  # to avoid classifier size mismatch from from_pretrained.
)

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['projector.weight', 'projector.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# freeze all layers to begin with
for param in hubert_model.parameters():
    param.requires_grad = False

# freeze two encoder layers
layers_freeze_num = 2
n_layers = (
    4 + layers_freeze_num * 16
)  # 4 refers to projector and classifier's weights and biases.
for name, param in list(hubert_model.named_parameters())[-n_layers:]:
    param.requires_grad = True

In [14]:
PROJECT_ROOT = "/content/drive/MyDrive/Data/Diss/"
dataset_config = {
  "LOADING_SCRIPT_FILES": os.path.join(PROJECT_ROOT, "common_voice.py"),
  "CONFIG_NAME": "clean",
  "DATA_DIR": os.path.join(PROJECT_ROOT, "trainn"),
  "CACHE_DIR": os.path.join(PROJECT_ROOT, "cache_crema"),
}
ds = load_dataset(
  dataset_config["LOADING_SCRIPT_FILES"],
  dataset_config["CONFIG_NAME"],
  data_dir=dataset_config["DATA_DIR"],
  cache_dir=dataset_config["CACHE_DIR"]
)
print(ds)

Generating train split: 0 examples [00:00, ? examples/s]

done 1
done 2
DatasetDict({
    train: Dataset({
        features: ['file', 'label'],
        num_rows: 3669
    })
})


In [15]:
# CONVERING RAW AUDIO TO ARRAYS
ds = ds.map(
        lambda x: {
            "array": librosa.load(x["file"], sr=16000, mono=False)[0]
        },
        num_proc=2,
    )

Map (num_proc=2):   0%|          | 0/3669 [00:00<?, ? examples/s]

In [16]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'array'],
        num_rows: 3669
    })
})


In [17]:
# LABEL TO ID
ds = ds.class_encode_column("label")

Casting to class labels:   0%|          | 0/3669 [00:00<?, ? examples/s]

In [18]:
# PROCESS THE DATASET TO THE FORMAT EXPECTED BY THE MODEL FOR TRAINING

INPUT_FIELD = "input_values"
LABEL_FIELD = "labels"

def prepare_dataset(batch, feature_extractor):
    audio_arr = batch["array"]
    input = feature_extractor(
        audio_arr, sampling_rate=16000, padding=True, return_tensors="pt"
    )

    batch[INPUT_FIELD] = input.input_values[0]
    batch[LABEL_FIELD] = batch[
        "label"
    ]  # colname MUST be labels as Trainer will look for it by default

    return batch

In [19]:
# APPLY THE DATA PREP USING FEATURE EXTRACTOR TO ALL EXAMPLES
ds = ds.map(
    prepare_dataset,
    fn_kwargs={"feature_extractor": feature_extractor},
    num_proc=2,
)
logging.info("Finished extracting features from audio arrays.")

Map (num_proc=2):   0%|          | 0/3669 [00:00<?, ? examples/s]

In [20]:
# INTRODUCE TRAIN TEST VAL SPLITS

# 90% train, 10% test + validation
train_testvalid = ds["train"].train_test_split(shuffle=True, test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'val': test_valid['train']})

In [21]:
trainer_config = {
  "OUTPUT_DIR": "results",
  "TRAIN_EPOCHS": 3,
  "TRAIN_BATCH_SIZE": 8,
  "EVAL_BATCH_SIZE": 8,
  "GRADIENT_ACCUMULATION_STEPS": 4,
  "WARMUP_STEPS": 500,
  "DECAY": 0.01,
  "LOGGING_STEPS": 10,
  "MODEL_DIR": "models/test-hubert-model",
  "SAVE_STEPS": 100
}

# Fine-Tuning with Trainer
training_args = TrainingArguments(
    output_dir=trainer_config["OUTPUT_DIR"],  # output directory
    gradient_accumulation_steps=trainer_config[
        "GRADIENT_ACCUMULATION_STEPS"
    ],  # accumulate the gradients before running optimization step
    num_train_epochs=trainer_config[
        "TRAIN_EPOCHS"
    ],  # total number of training epochs
    per_device_train_batch_size=trainer_config[
        "TRAIN_BATCH_SIZE"
    ],  # batch size per device during training
    per_device_eval_batch_size=trainer_config[
        "EVAL_BATCH_SIZE"
    ],  # batch size for evaluation
    warmup_steps=trainer_config[
        "WARMUP_STEPS"
    ],  # number of warmup steps for learning rate scheduler
    save_steps=trainer_config["SAVE_STEPS"], # save checkpoint every 100 steps
    weight_decay=trainer_config["DECAY"],  # strength of weight decay
    logging_steps=trainer_config["LOGGING_STEPS"],
    evaluation_strategy="epoch",  # report metric at end of each epoch
    report_to="none",  # enable logging to W&B
)

In [22]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

import torch
from transformers import Wav2Vec2Processor

INPUT_FIELD = "input_values"
LABEL_FIELD = "labels"


@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, examples: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:

        input_features = [
            {INPUT_FIELD: example[INPUT_FIELD]} for example in examples
        ]  # example is basically row0, row1, etc...
        labels = [example[LABEL_FIELD] for example in examples]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        batch[LABEL_FIELD] = torch.tensor(labels)

        return batch

In [23]:
# DEFINE DATA COLLATOR - TO PAD TRAINING BATCHES DYNAMICALLY
data_collator = DataCollatorCTCWithPadding(
    processor=feature_extractor, padding=True
)


def compute_metrics(eval_pred):
    # DEFINE EVALUATION METRIC
    compute_accuracy_metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return compute_accuracy_metric.compute(predictions=predictions, references=labels)

# START TRAINING
trainer = Trainer(
    model=hubert_model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=ds["train"],  # training dataset
    eval_dataset=ds["val"],  # evaluation dataset
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: array, file. If array, file are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3302
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 309


Epoch,Training Loss,Validation Loss,Accuracy
0,1.944100,1.878982,0.333333
1,1.796400,1.752367,0.333333
2,1.689800,1.732284,0.333333


Saving model checkpoint to results/checkpoint-100
Configuration saved in results/checkpoint-100/config.json
Model weights saved in results/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: array, file. If array, file are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 183
  Batch size = 8


Saving model checkpoint to results/checkpoint-200
Configuration saved in results/checkpoint-200/config.json
Model weights saved in results/checkpoint-200/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: array, file. If array, file are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 183
  Batch size = 8
Saving model checkpoint to results/checkpoint-300
Configuration saved in results/checkpoint-300/config.json
Model weights saved in results/checkpoint-300/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: array, file. If array, file are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Nu

TrainOutput(global_step=309, training_loss=1.8684974411158886, metrics={'train_runtime': 1047.743, 'train_samples_per_second': 9.455, 'train_steps_per_second': 0.295, 'total_flos': 5.866176416850248e+17, 'train_loss': 1.8684974411158886, 'epoch': 3.0})

In [25]:
test_results = trainer.predict(ds["test"])

The following columns in the test set don't have a corresponding argument in `HubertForSequenceClassification.forward` and have been ignored: array, file. If array, file are not expected by `HubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 184
  Batch size = 8
